In [1]:
import os
import sys
import time
import numpy as np
import scipy.sparse as sp
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS

In [3]:
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits, evaluate_preds
from ggcn import GaussianGraphConvolution

# Data

In [4]:
X, A, y = load_data('data/cora/', dataset='cora')
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [8]:
support = 1
graph = [X, A]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

W1110 16:46:17.470891 140571322152768 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1110 16:46:17.479721 140571322152768 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:515: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.



In [9]:
X_in = Input(shape=(X.shape[1],))

W1110 16:46:17.484791 140571322152768 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



# GCN (baseline)

In [10]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(32, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

W1110 16:46:17.502196 140571322152768 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1110 16:46:17.506567 140571322152768 deprecation.py:506] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1110 16:46:17.515767 140571322152768 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [11]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

W1110 16:46:17.558493 140571322152768 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 32)           45888       dropout_1[0][0]                  
                                                                 input_1[0][0]                    
__________

In [13]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

W1110 16:46:17.640092 140571322152768 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 0001 train_loss= 1.9307 train_acc= 0.3857 val_loss= 1.9326 val_acc= 0.3700 time= 1.0964
Epoch: 0002 train_loss= 1.9139 train_acc= 0.3143 val_loss= 1.9175 val_acc= 0.3533 time= 0.0173
Epoch: 0003 train_loss= 1.8957 train_acc= 0.3143 val_loss= 1.9016 val_acc= 0.3533 time= 0.0171
Epoch: 0004 train_loss= 1.8761 train_acc= 0.3286 val_loss= 1.8847 val_acc= 0.3600 time= 0.0166
Epoch: 0005 train_loss= 1.8554 train_acc= 0.3286 val_loss= 1.8666 val_acc= 0.3600 time= 0.0181
Epoch: 0006 train_loss= 1.8348 train_acc= 0.3286 val_loss= 1.8488 val_acc= 0.3600 time= 0.0204
Epoch: 0007 train_loss= 1.8144 train_acc= 0.3286 val_loss= 1.8312 val_acc= 0.3567 time= 0.0170
Epoch: 0008 train_loss= 1.7942 train_acc= 0.3286 val_loss= 1.8144 val_acc= 0.3567 time= 0.0185
Epoch: 0009 train_loss= 1.7749 train_acc= 0.3286 val_loss= 1.7987 val_acc= 0.3567 time= 0.0175
Epoch: 0010 train_loss= 1.7573 train_acc= 0.3286 val_loss= 1.7847 val_acc= 0.3567 time= 0.0188
Epoch: 0011 train_loss= 1.7413 train_acc= 0.3214 v

Epoch: 0097 train_loss= 0.6360 train_acc= 0.9357 val_loss= 0.9461 val_acc= 0.7900 time= 0.0203
Epoch: 0098 train_loss= 0.6292 train_acc= 0.9357 val_loss= 0.9424 val_acc= 0.8000 time= 0.0189
Epoch: 0099 train_loss= 0.6237 train_acc= 0.9357 val_loss= 0.9397 val_acc= 0.8033 time= 0.0165
Epoch: 0100 train_loss= 0.6183 train_acc= 0.9357 val_loss= 0.9366 val_acc= 0.8000 time= 0.0170
Epoch: 0101 train_loss= 0.6128 train_acc= 0.9357 val_loss= 0.9334 val_acc= 0.8000 time= 0.0172
Epoch: 0102 train_loss= 0.6068 train_acc= 0.9357 val_loss= 0.9291 val_acc= 0.7967 time= 0.0175
Epoch: 0103 train_loss= 0.6009 train_acc= 0.9357 val_loss= 0.9244 val_acc= 0.8000 time= 0.0173
Epoch: 0104 train_loss= 0.5953 train_acc= 0.9357 val_loss= 0.9190 val_acc= 0.8000 time= 0.0176
Epoch: 0105 train_loss= 0.5902 train_acc= 0.9357 val_loss= 0.9140 val_acc= 0.7933 time= 0.0167
Epoch: 0106 train_loss= 0.5857 train_acc= 0.9357 val_loss= 0.9088 val_acc= 0.7867 time= 0.0168
Epoch: 0107 train_loss= 0.5810 train_acc= 0.9357 v

Epoch: 0193 train_loss= 0.3468 train_acc= 0.9714 val_loss= 0.7433 val_acc= 0.8200 time= 0.0183
Epoch: 0194 train_loss= 0.3469 train_acc= 0.9714 val_loss= 0.7439 val_acc= 0.8100 time= 0.0189
Epoch: 0195 train_loss= 0.3456 train_acc= 0.9714 val_loss= 0.7419 val_acc= 0.8067 time= 0.0182
Epoch: 0196 train_loss= 0.3419 train_acc= 0.9714 val_loss= 0.7373 val_acc= 0.8100 time= 0.0172
Epoch: 0197 train_loss= 0.3384 train_acc= 0.9786 val_loss= 0.7336 val_acc= 0.8100 time= 0.0167
Epoch: 0198 train_loss= 0.3373 train_acc= 0.9786 val_loss= 0.7318 val_acc= 0.8233 time= 0.0179
Epoch: 0199 train_loss= 0.3380 train_acc= 0.9786 val_loss= 0.7321 val_acc= 0.8233 time= 0.0201
Epoch: 0200 train_loss= 0.3378 train_acc= 0.9786 val_loss= 0.7342 val_acc= 0.8300 time= 0.0179


In [14]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.7619
accuracy = 0.8280


# Gaussian Graph Convolution

In [15]:
H = Dropout(0.5)(X_in)
H = GaussianGraphConvolution(16, A.shape[0], 
    is_first=True, 
    activation='relu', 
    kernel_regularizer=l2(5e-4)
)([H]+G)
Y = GaussianGraphConvolution(y.shape[1], A.shape[0], 
    is_last=True,
    activation='softmax'
)(H+G)

W1110 16:46:22.247374 140571322152768 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/ggcl.py:163: Normal.__init__ (from tensorflow.python.ops.distributions.normal) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
W1110 16:46:22.248535 140571322152768 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/distributions/normal.py:160: Distribution.__init__ (from tensorflow.python.ops.distributions.distribution) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should updat

In [16]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
gaussian_graph_convolution_1 (G [(None, 16), (None,  45856       dropout_3[0][0]                  
                                                                 input_1[0][0]                    
__________

In [18]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

Epoch: 0001 train_loss= 1.9384 train_acc= 0.4214 val_loss= 1.9405 val_acc= 0.3700 time= 0.3997
Epoch: 0002 train_loss= 1.9316 train_acc= 0.4857 val_loss= 1.9346 val_acc= 0.4733 time= 0.0164
Epoch: 0003 train_loss= 1.9212 train_acc= 0.6000 val_loss= 1.9283 val_acc= 0.5133 time= 0.0165
Epoch: 0004 train_loss= 1.9109 train_acc= 0.6643 val_loss= 1.9207 val_acc= 0.5767 time= 0.0161
Epoch: 0005 train_loss= 1.9014 train_acc= 0.7143 val_loss= 1.9134 val_acc= 0.5700 time= 0.0170
Epoch: 0006 train_loss= 1.8883 train_acc= 0.6929 val_loss= 1.9039 val_acc= 0.6000 time= 0.0169
Epoch: 0007 train_loss= 1.8762 train_acc= 0.7071 val_loss= 1.8952 val_acc= 0.5633 time= 0.0172
Epoch: 0008 train_loss= 1.8626 train_acc= 0.7071 val_loss= 1.8867 val_acc= 0.5500 time= 0.0170
Epoch: 0009 train_loss= 1.8475 train_acc= 0.7143 val_loss= 1.8758 val_acc= 0.5667 time= 0.0170
Epoch: 0010 train_loss= 1.8327 train_acc= 0.7429 val_loss= 1.8630 val_acc= 0.5633 time= 0.0171
Epoch: 0011 train_loss= 1.8178 train_acc= 0.7286 v

Epoch: 0099 train_loss= 0.2121 train_acc= 0.9857 val_loss= 0.6586 val_acc= 0.8200 time= 0.0165
Epoch: 0100 train_loss= 0.2063 train_acc= 0.9857 val_loss= 0.6538 val_acc= 0.8200 time= 0.0178
Epoch: 0101 train_loss= 0.2013 train_acc= 0.9857 val_loss= 0.6523 val_acc= 0.8200 time= 0.0163
Epoch: 0102 train_loss= 0.1948 train_acc= 0.9857 val_loss= 0.6479 val_acc= 0.8200 time= 0.0161
Epoch: 0103 train_loss= 0.1906 train_acc= 0.9857 val_loss= 0.6446 val_acc= 0.8200 time= 0.0160
Epoch: 0104 train_loss= 0.1852 train_acc= 0.9857 val_loss= 0.6421 val_acc= 0.8233 time= 0.0178
Epoch: 0105 train_loss= 0.1804 train_acc= 0.9857 val_loss= 0.6400 val_acc= 0.8200 time= 0.0168
Epoch: 0106 train_loss= 0.1758 train_acc= 0.9857 val_loss= 0.6366 val_acc= 0.8167 time= 0.0171
Epoch: 0107 train_loss= 0.1711 train_acc= 0.9857 val_loss= 0.6351 val_acc= 0.8200 time= 0.0183
Epoch: 0108 train_loss= 0.1676 train_acc= 0.9857 val_loss= 0.6328 val_acc= 0.8167 time= 0.0172
Epoch: 0109 train_loss= 0.1625 train_acc= 0.9857 v

In [19]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.6309
accuracy = 0.8090


# Attacks